In [1]:
from asteroid_interp import interp

In [2]:
eval_test = \
'''
let a = 1.
let b = 1.
let v1 = eval 
    "('integer', 
        int(state.symbol_table.lookup_sym('a')[1]) + 
        int(state.symbol_table.lookup_sym('b')[1]))".

print v1.
'''
interp(eval_test)


(seq 
  |(assign 
  |  |(id a) 
  |  |(integer 1)) 
  |(seq 
  |  |(assign 
  |  |  |(id b) 
  |  |  |(integer 1)) 
  |  |(seq 
  |  |  |(assign 
  |  |  |  |(id v1) 
  |  |  |  |(eval ('integer', 
        int(state.symbol_table.lookup_sym('a')[1]) + 
        int(state.symbol_table.lookup_sym('b')[1])))) 
  |  |  |(seq 
  |  |  |  |(print 
  |  |  |  |  |(id v1) 
  |  |  |  |  |(file-name 
  |  |  |  |  |  |(nil))) 
  |  |  |  |(nil)))))
None
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'a': [('integer', '1')],
  'b': [('integer', '1')],
  'v1': [None]}]


In [3]:
attach = \
'''
-- this function implements the behavior for the '+' operator for ints, reals, and strings.

function add with a,b do
    let v1 = eval 
"
global __retval__ # return register from 'eval' calls
type = promote(state.symbol_table.lookup_sym('a')[0], 
               state.symbol_table.lookup_sym('b')[0])
if type == 'real':
    __retval__ = ('real', 
            float(state.symbol_table.lookup_sym('a')[1]) + 
            float(state.symbol_table.lookup_sym('b')[1]))
elif type == 'integer':
    __retval__ = ('integer', 
            int(state.symbol_table.lookup_sym('a')[1]) + 
            int(state.symbol_table.lookup_sym('b')[1]))
elif type == 'string':
    __retval__ = ('string', 
            str(state.symbol_table.lookup_sym('a')[1])+
            str(state.symbol_table.lookup_sym('b')[1]))
else:
    raise ValueError('unsupported type in add')
".
        
    return v1
end function

-- attach add to __plus__.

let x = 1.2 + 2 .
print x.
print "the output is: " + x.
'''
interp(attach, tree_dump=False, symtab_dump=False, exceptions=True)

('__plus__', ('real', '1.2'), ('integer', '2'))
('__plus__', ('string', 'the output is: '), ('__plus__', ('real', '1.2'), ('integer', '2')))


In [4]:
interp("let x = 1 + 2. print x.")


(seq 
  |(assign 
  |  |(id x) 
  |  |(__plus__ 
  |  |  |(integer 1) 
  |  |  |(integer 2))) 
  |(seq 
  |  |(print 
  |  |  |(id x) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))
('__plus__', ('integer', '1'), ('integer', '2'))
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'x': [('__plus__', ('integer', '1'), ('integer', '2'))]}]


In [5]:
interp("let x = 1.3 . print x.")


(seq 
  |(assign 
  |  |(id x) 
  |  |(real 1.3)) 
  |(seq 
  |  |(print 
  |  |  |(id x) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))
('real', '1.3')
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'x': [('real', '1.3')]}]


In [6]:
interp("let 1,y = 1,2. print 1,y.")


(seq 
  |(assign 
  |  |(list 
  |  |  |[ 
  |  |  |  |(integer 1) 
  |  |  |  |(id y)]) 
  |  |(list 
  |  |  |[ 
  |  |  |  |(integer 1) 
  |  |  |  |(integer 2)])) 
  |(seq 
  |  |(print 
  |  |  |(list 
  |  |  |  |[ 
  |  |  |  |  |(integer 1) 
  |  |  |  |  |(id y)]) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))
('list', [('integer', '1'), ('integer', '2')])
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'y': [('integer', '2')]}]


In [7]:
interp("let x,2 = 1,2. print x,2.")


(seq 
  |(assign 
  |  |(list 
  |  |  |[ 
  |  |  |  |(id x) 
  |  |  |  |(integer 2)]) 
  |  |(list 
  |  |  |[ 
  |  |  |  |(integer 1) 
  |  |  |  |(integer 2)])) 
  |(seq 
  |  |(print 
  |  |  |(list 
  |  |  |  |[ 
  |  |  |  |  |(id x) 
  |  |  |  |  |(integer 2)]) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))
('list', [('integer', '1'), ('integer', '2')])
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'x': [('integer', '1')]}]


In [8]:
interp("let s = 1,2. let x,y = s. print x,y.")


(seq 
  |(assign 
  |  |(id s) 
  |  |(list 
  |  |  |[ 
  |  |  |  |(integer 1) 
  |  |  |  |(integer 2)])) 
  |(seq 
  |  |(assign 
  |  |  |(list 
  |  |  |  |[ 
  |  |  |  |  |(id x) 
  |  |  |  |  |(id y)]) 
  |  |  |(id s)) 
  |  |(seq 
  |  |  |(print 
  |  |  |  |(list 
  |  |  |  |  |[ 
  |  |  |  |  |  |(id x) 
  |  |  |  |  |  |(id y)]) 
  |  |  |  |(file-name 
  |  |  |  |  |(nil))) 
  |  |  |(nil))))
('list', [('integer', '1'), ('integer', '2')])
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  's': [('list', [('integer', '1'), ('integer', '2')])],
  'x': [('integer', '1')],
  'y': [('integer', '2')]}]


In [9]:
interp("function ident with n do return n end function print ident(2).")


(seq 
  |(assign 
  |  |(id ident) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(nil))))) 
  |(seq 
  |  |(print 
  |  |  |(juxta 
  |  |  |  |(id ident) 
  |  |  |  |(juxta 
  |  |  |  |  |(integer 2) 
  |  |  |  |  |(nil))) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))
('integer', '2')
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'ident': [('function',
             ('body-list',
              ('seq',
               ('body',
                ('pattern', ('id', 'n')),
                ('stmt-list', ('seq', ('return', ('id', 'n')), ('nil',)))),
               ('nil',))))]}]


In [10]:
interp("function ident with n do return n end function print ident ident 2 .", exceptions=True)


(seq 
  |(assign 
  |  |(id ident) 
  |  |(function 
  |  |  |(body-list 
  |  |  |  |(seq 
  |  |  |  |  |(body 
  |  |  |  |  |  |(pattern 
  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |(stmt-list 
  |  |  |  |  |  |  |(seq 
  |  |  |  |  |  |  |  |(return 
  |  |  |  |  |  |  |  |  |(id n)) 
  |  |  |  |  |  |  |  |(nil)))) 
  |  |  |  |  |(nil))))) 
  |(seq 
  |  |(print 
  |  |  |(juxta 
  |  |  |  |(id ident) 
  |  |  |  |(juxta 
  |  |  |  |  |(id ident) 
  |  |  |  |  |(juxta 
  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |(nil)))) 
  |  |  |(file-name 
  |  |  |  |(nil))) 
  |  |(nil)))
('integer', '2')
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'ident': [('function',
             ('body-list',
              ('seq',
               ('body',
                ('pattern', ('id', 'n')),
                ('stmt-list', ('seq', ('return', ('id', 'n')), ('nil',)))),
               ('nil',))))]}]


In [11]:
interp("let _, x = [1], 2.")


(seq 
  |(assign 
  |  |(list 
  |  |  |[ 
  |  |  |  |(id _) 
  |  |  |  |(id x)]) 
  |  |(list 
  |  |  |[ 
  |  |  |  |(list 
  |  |  |  |  |[ 
  |  |  |  |  |  |(integer 1)]) 
  |  |  |  |(integer 2)])) 
  |(nil))
Symbol Table Dump:
[{'__minus__': [('constructor', ('arity', 2))],
  '__plus__': [('constructor', ('arity', 2))],
  'x': [('integer', '2')]}]


In [12]:
interp("let x = y[0][1].")


(seq 
  |(assign 
  |  |(id x) 
  |  |(juxta 
  |  |  |(id y) 
  |  |  |(juxta 
  |  |  |  |(list 
  |  |  |  |  |[ 
  |  |  |  |  |  |(integer 0)]) 
  |  |  |  |(juxta 
  |  |  |  |  |(list 
  |  |  |  |  |  |[ 
  |  |  |  |  |  |  |(integer 1)]) 
  |  |  |  |  |(nil))))) 
  |(nil))
Error: y is not defined
